# Objetos na Memória (parte 3)

Este notebook contém exemplos do livro [_Fluent Python, Second Edition_](https://www.fluentpython.com/) .

## Afinal, tuplas são mesmo imutáveis?

A melhor resposta é: **depende**.

Uma tupla é um _container_: uma estrutura de dados que contém referências a outros objetos.
Outros containers são listas, dicts, sets, e filas.

> Em contraste, `str`, `bytes` e `array` são _coleções simples_: armazenam diretamente seus dados, e não referências a outros objetos.

Em uma tupla, as referências são imutáveis.
Ou seja, uma vez criada a tupla, ela sempre apontará para a mesma sequência de objetos.

Porém, se qualquer objeto contido na tupla for mutável,
e seu valor for alterado, isso mudará também o valor da tupla!

![tupla-com-lista](fig2-4.png)

In [1]:
a = (10, 'alpha', [1, 2])
b = (10, 'alpha', [1, 2])
a == b  # valores iguais

True

In [2]:
a is b  # identidades diferentes

False

In [3]:
id(b[-1])  # id da lista que é o último item da tupla b

4437608960

In [4]:
b[-1].append(99)  # alterando a lista t1[-1]
a == b  # agora as tuplas tem valores diferentes

False

In [5]:
b

(10, 'alpha', [1, 2, 99])

In [6]:
id(b[-1])  # mas o id de b[-1] não mudou

4437608960

👉 **RESUMO:** o que nunca muda em uma tupla são as referências que ela contém. Mas se o valor de um item da tupla mudar, isso muda o valor da tupla.

Em outras palavras: a identidade dos itens em uma tupla nunca muda, mas seus valores podem mudar, dependendo do tipo do item.

## Cópias rasas ou profundas?

Ao copiar um container, é preciso tomar uma decisão: os itens do novo container devem ser cópias dos itens originais, ou podem ser referências para os mesmos itens?

No segundo caso, os itens são compartilhados entre os conteiners.
Isso economiza memória, mas pode criar problemas.
Porque se um objeto está incluído em dois conteiners, está acontecendo apelidamento (*aliasing*).
Se todos os objetos compartilhados forem imutáveis, não tem problema.

Mas se um objeto compartilhado é mutável, e ele for alterado através de um dos containers,
a mudança vai aparecer no outro container.
Isso pode ser desejável ou não.

Quando a cópia compartilha itens de mesma identidade com o original, dizemos que é uma *cópia rasa* (shallow copy).<br>
Se todos os itens da cópia forem copiados também,
é uma *cópia profunda* (*deep copy*).

Muitas vezes a forma mais fácil de criar uma cópia rasa de um container é usar seu costrutor:

In [7]:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = list(l1)
l2

[3, [55, 44], (7, 8, 9)]

In [8]:
l1[1] is l2[1]

True

A linha acima prova que `l2` é uma cópia rasa de `l1`, porque a lista `[55, 44]` é compartilhada por `l1` e `l2`.
No caso de listas e outras sequências mutáveis, a operação `l2 = l1[:]` também cria uma cópia rasa.

## Crie cópias profundas com `deepcopy`

O [módulo `copy`](https://docs.python.org/pt-br/3/library/copy.html) de Python fornece as funções `copy` e `deepcopy`, que servem para fazer cópias rasas ou profundas de objetos arbitrários.

Para ilustrar esse ponto, vamos estudar uma classe simples que representa
um ônibus com uma lista de nomes de passageiros.

In [9]:
class Bus:

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)  

Além de `__init__`, a classe tem dois métodos:

* `pick` recebe um nome e o coloca na lista de passageiros;
* `drop` recebe um nome e o retira da lista, se estiver lá; caso contrário, ocorre uma exceção.

Agora vamos fazer experimentos com uma cópia rasa e uma cópia profunda de uma instância de `Bus`.

In [10]:
from copy import copy, deepcopy
bus1 = Bus(['Alice', 'Beto', 'Clara', 'Davi'])
bus2 = copy(bus1)
bus3 = deepcopy(bus1)
id(bus1), id(bus2), id(bus3)  # três ônibus distintos

(4437610448, 4437610640, 4437593872)

In [11]:
bus1.drop('Beto')
bus2.passengers

['Alice', 'Clara', 'Davi']

Note que `'Beto'` foi tirado de `bus1` mas sumiu de `bus2`, porque esta é uma cópia rasa de `bus1`, então elas compartilham a mesma lista de passageiros, apesar de serem instâncias separadas da classe `Bus`.

In [12]:
bus1.passengers is bus2.passengers

True

Mas `bus3` é uma cópia profunda de `bus1`, portanto elas não compartilham a mesma lista de passageiros.
Por isso `'Beto'` não desaparece de `bus3` quando ele é retirado de `bus1`.

In [13]:
bus3.passengers

['Alice', 'Beto', 'Clara', 'Davi']

Não existe uma resposta padrão para decidir entre cópias rasas ou profundas.
É uma decisão caso a caso.

Nem sempre uma cópia profunda é necessária ou correta, e pode até ser impossível se um dos atributos do objeto a ser copiado é um recurso que existe fora da memória do seu programa, como um arquivo ou uma conexão de rede—que são gerenciados pelo sistema operational.

É possível controlar precisamente como as instâncias de uma classe que você criou vão funcionar com as funções `copy` e `deepcopy`, se você implementar os métodos especiais `__copy__` e `__deepcopy__`,
como descrito na [documentação do módulo](https://docs.python.org/pt-br/3/library/copy.html) `copy`.